In [1]:
# import QGRID para mejor visualizacion de los datos
# los pptos sin abonos se deben cancelar en ClinicCloud, cosa q no generen alertas. O q el paciente tenga saldo a favor
# Debemos Generar un archivo identificandolos. Las ff´s deben tener algun ingreso de dinero para no cancelarlas.
# Los pptos aceptados hoy pueden haber sido elaborados en una fecha anterior. Su fecha se debe actualizar a la de hoy, 
# ya sea en ClinicCloud o en el reporte (mejor - 'fecha_inicio')

import numpy as np
import pandas as pd
import time

pd.options.display.max_columns = 999
pd.options.display.max_rows = 999
pd.set_option('display.max_colwidth', -1)

In [2]:
archivo_PPE = 'Produccion_pendiente_exigible_con_tratamientos_Dentix_Calle_90.csv'
archivo_crono = 'crono_Dentix_Calle_90_1.csv'

In [3]:
crono_df = pd.read_csv(archivo_crono, encoding = 'unicode_escape', sep=',')
PPE = pd.read_csv(archivo_PPE, encoding = 'unicode_escape', sep=',')

#Eliminación de Puntos y Revisiones del PPE
PPE.set_index('Servicio', inplace=True)
PPE.drop(index='Puntos y Revisiones.  ',inplace=True)
PPE.reset_index(inplace=True)
PPE['duplicated'] = PPE.duplicated()
PPE.set_index('duplicated',inplace=True)
PPE.drop(True,inplace=True)
PPE.reset_index(inplace=True)
PPE.drop(columns=['duplicated'],inplace=True)
PPE

,Servicio,Clínica,Fecha Pres.,Nº Historia,Presupuesto,Descuento,Teléfono Paciente,Doctor,Especialidad,Explicado,Importe Pres.,Pagado Pres.,% Pagado,% Realizado,Producidos,Pendientes Producción,Próxima Cita,Fecha Última Cita Prod
0,RETENEDOR FIJO 3X3 EN ALAMBRE. Pieza: 11,Dentix Calle 90,22/09/2018,BG025054,Nº 4848 - Carlos Julio Alba Martinez,0.0,79902188,JOHANNA LISSETE MENDOZA DUARTE,ORTODONCIA Y REPARACIONES,ALISSON SARAY MORA CASTAÑEDA,150000,0,0.00%,0.00%,0,150000.0,22/09/2018,Sin Cita Producida
1,RETENEDOR FIJO 3X3 EN ALAMBRE. Pieza: 31,Dentix Calle 90,22/09/2018,BG025054,Nº 4848 - Carlos Julio Alba Martinez,0.0,79902188,JOHANNA LISSETE MENDOZA DUARTE,ORTODONCIA Y REPARACIONES,ALISSON SARAY MORA CASTAÑEDA,150000,0,0.00%,0.00%,0,150000.0,22/09/2018,Sin Cita Producida
2,RETENEDOR ESSIX O ESTETICOS. Pieza: 11,Dentix Calle 90,22/09/2018,BG025054,Nº 4848 - Carlos Julio Alba Martinez,0.0,79902188,JOHANNA LISSETE MENDOZA DUARTE,ORTODONCIA Y REPARACIONES,ALISSON SARAY MORA CASTAÑEDA,150000,0,0.00%,0.00%,0,150000.0,22/09/2018,Sin Cita Producida
3,RETENEDOR FIJO 3X3 EN ALAMBRE. Pieza: 11,Dentix Calle 90,22/09/2018,BG058253,Nº 4850 - CARLOS JULIO ALBA,0.0,79902188,JOHANNA LISSETE MENDOZA DUARTE,ORTODONCIA Y REPARACIONES,ALISSON SARAY MORA CASTAÑEDA,150000,0,0.00%,0.00%,0,150000.0,13/10/2018,Sin Cita Producida
4,RETENEDOR FIJO 3X3 EN ALAMBRE. Pieza: 31,Dentix Calle 90,22/09/2018,BG058253,Nº 4850 - CARLOS JULIO ALBA,0.0,79902188,JOHANNA LISSETE MENDOZA DUARTE,ORTODONCIA Y REPARACIONES,ALISSON SARAY MORA CASTAÑEDA,150000,0,0.00%,0.00%,0,150000.0,13/10/2018,Sin Cita Producida
5,RETENEDOR ESSIX O ESTETICOS. Pieza: 11,Dentix Calle 90,22/09/2018,BG058253,Nº 4850 - CARLOS JULIO ALBA,0.0,79902188,JOHANNA LISSETE MENDOZA DUARTE,ORTODONCIA Y REPARACIONES,ALISSON SARAY MORA CASTAÑEDA,150000,0,0.00%,0.00%,0,150000.0,13/10/2018,Sin Cita Producida
6,DETARTRAJE POR ARCADA.,Dentix Calle 90,22/09/2018,BG058253,Nº 4851 - CARLOS JULIO ALBA,0.0,79902188,LAURA ALEJANDRA RONCANCIO GARCIA,OPERATORIA GENERAL,ALISSON SARAY MORA CASTAÑEDA,97520,0,0.00%,0.00%,0,97500.0,13/10/2018,Sin Cita Producida
7,PROFILAXIS. Pieza: 11,Dentix Calle 90,22/09/2018,BG058253,Nº 4851 - CARLOS JULIO ALBA,0.0,79902188,LAURA ALEJANDRA RONCANCIO GARCIA,OPERATORIA GENERAL,ALISSON SARAY MORA CASTAÑEDA,40000,0,0.00%,0.00%,0,40000.0,13/10/2018,Sin Cita Producida
8,ALARGAMIENTO CORONAL (POR PIEZA).,Dentix Calle 90,22/09/2018,BG056466,Nº 4854 - Yobany Garcia Franco,0.0,3116218366,LAURA ALEJANDRA RONCANCIO GARCIA,CIRUJIA ORAL Y MAXILOFACIAL,ALISSON SARAY MORA CASTAÑEDA,175000,0,0.00%,0.00%,0,175000.0,25/09/2018,Sin Cita Producida
9,RESINA FOTOPOLIMERIZADO 1 SUP POSTERIOR Y ANTERIOR. Pieza: 15,Dentix Calle 90,22/09/2018,BG056466,Nº 4854 - Yobany Garcia Franco,0.0,3116218366,LAURA ALEJANDRA RONCANCIO GARCIA,OPERATORIA GENERAL,ALISSON SARAY MORA CASTAÑEDA,79000,0,0.00%,0.00%,0,79000.0,25/09/2018,Sin Cita Producida


In [4]:
print('REGISTROS CRONOGRAMA:',crono_df.count()[0])
print('REGISTROS PPE:',PPE.count()[0])

REGISTROS CRONOGRAMA: 992
REGISTROS PPE: 1056


In [5]:
#CRUCE PPE - CRONO: en este punto revisar que cada ttmto del PPE tenga su cronograma. Sinó, sacar los pptos q faltan por crono.

pptos_faltantes = pd.merge(PPE,crono_df,how='left',left_on=['Clínica','Nº Historia','Presupuesto','Servicio'], right_on=['Clínica','Nº Historia','Presupuesto','Servicio'])
pptos_faltantes['Semana'].fillna(0,inplace=True)
pptos_faltantes.set_index('Semana',inplace=True)
pptos_faltantes = pptos_faltantes.loc[0]

pptos_faltantes

,Servicio,Clínica,Fecha Pres._x,Nº Historia,Presupuesto,Descuento,Teléfono Paciente,Doctor_x,Especialidad,Explicado_x,Importe Pres.,Pagado Pres.,% Pagado,% Realizado,Producidos,Pendientes Producción,Próxima Cita,Fecha Última Cita Prod,Fecha Pres._y,Cantidad Presupuestada,Doctor_y,Explicado_y,ttmto_homol,pieza,fecha_inicio
Semana,,,,,,,,,,,,,,,,,,,,,,,,,
0.0,RETENEDOR FIJO 3X3 EN ALAMBRE. Pieza: 11,Dentix Calle 90,22/09/2018,BG025054,Nº 4848 - Carlos Julio Alba Martinez,0.0,79902188,JOHANNA LISSETE MENDOZA DUARTE,ORTODONCIA Y REPARACIONES,ALISSON SARAY MORA CASTAÑEDA,150000,0,0.00%,0.00%,0,150000.0,22/09/2018,Sin Cita Producida,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0.0,RETENEDOR FIJO 3X3 EN ALAMBRE. Pieza: 31,Dentix Calle 90,22/09/2018,BG025054,Nº 4848 - Carlos Julio Alba Martinez,0.0,79902188,JOHANNA LISSETE MENDOZA DUARTE,ORTODONCIA Y REPARACIONES,ALISSON SARAY MORA CASTAÑEDA,150000,0,0.00%,0.00%,0,150000.0,22/09/2018,Sin Cita Producida,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0.0,RETENEDOR ESSIX O ESTETICOS. Pieza: 11,Dentix Calle 90,22/09/2018,BG025054,Nº 4848 - Carlos Julio Alba Martinez,0.0,79902188,JOHANNA LISSETE MENDOZA DUARTE,ORTODONCIA Y REPARACIONES,ALISSON SARAY MORA CASTAÑEDA,150000,0,0.00%,0.00%,0,150000.0,22/09/2018,Sin Cita Producida,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0.0,RETENEDOR FIJO 3X3 EN ALAMBRE. Pieza: 11,Dentix Calle 90,22/09/2018,BG058253,Nº 4850 - CARLOS JULIO ALBA,0.0,79902188,JOHANNA LISSETE MENDOZA DUARTE,ORTODONCIA Y REPARACIONES,ALISSON SARAY MORA CASTAÑEDA,150000,0,0.00%,0.00%,0,150000.0,13/10/2018,Sin Cita Producida,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0.0,RETENEDOR FIJO 3X3 EN ALAMBRE. Pieza: 31,Dentix Calle 90,22/09/2018,BG058253,Nº 4850 - CARLOS JULIO ALBA,0.0,79902188,JOHANNA LISSETE MENDOZA DUARTE,ORTODONCIA Y REPARACIONES,ALISSON SARAY MORA CASTAÑEDA,150000,0,0.00%,0.00%,0,150000.0,13/10/2018,Sin Cita Producida,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0.0,RETENEDOR ESSIX O ESTETICOS. Pieza: 11,Dentix Calle 90,22/09/2018,BG058253,Nº 4850 - CARLOS JULIO ALBA,0.0,79902188,JOHANNA LISSETE MENDOZA DUARTE,ORTODONCIA Y REPARACIONES,ALISSON SARAY MORA CASTAÑEDA,150000,0,0.00%,0.00%,0,150000.0,13/10/2018,Sin Cita Producida,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0.0,DETARTRAJE POR ARCADA.,Dentix Calle 90,22/09/2018,BG058253,Nº 4851 - CARLOS JULIO ALBA,0.0,79902188,LAURA ALEJANDRA RONCANCIO GARCIA,OPERATORIA GENERAL,ALISSON SARAY MORA CASTAÑEDA,97520,0,0.00%,0.00%,0,97500.0,13/10/2018,Sin Cita Producida,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0.0,PROFILAXIS. Pieza: 11,Dentix Calle 90,22/09/2018,BG058253,Nº 4851 - CARLOS JULIO ALBA,0.0,79902188,LAURA ALEJANDRA RONCANCIO GARCIA,OPERATORIA GENERAL,ALISSON SARAY MORA CASTAÑEDA,40000,0,0.00%,0.00%,0,40000.0,13/10/2018,Sin Cita Producida,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0.0,ALARGAMIENTO CORONAL (POR PIEZA).,Dentix Calle 90,22/09/2018,BG056466,Nº 4854 - Yobany Garcia Franco,0.0,3116218366,LAURA ALEJANDRA RONCANCIO GARCIA,CIRUJIA ORAL Y MAXILOFACIAL,ALISSON SARAY MORA CASTAÑEDA,175000,0,0.00%,0.00%,0,175000.0,25/09/2018,Sin Cita Producida,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
pptos_faltantes.reset_index(inplace=True)
pptos_faltantes['Cantidad Presupuestada'] = (pptos_faltantes['Producidos']+pptos_faltantes['Pendientes Producción'])/pptos_faltantes['Importe Pres.']
pptos_faltantes['Cantidad Presupuestada'] = np.ceil(pptos_faltantes['Cantidad Presupuestada'])
pptos_faltantes['Cantidad Presupuestada'].fillna(1,inplace=True) #las CSI traen vacío el campo 'Pendientes Produccion'.
# en realidad parece que las unicas cantidades presupuestadas q son necesarias son las de Mensualidades de ortodoncia.
# Para CSI, el %Realizado es el que nos sirve para calcular Cantidad Producida.
# la Cantidad Pendiente se puede utilizar para saber la fecha de colocación de un implante si hay ortodoncia.

pptos_faltantes['ttmto_homol'] = pptos_faltantes['Servicio'].str.slice(0,-13)
pptos_faltantes['ttmto_homol'] = pptos_faltantes['ttmto_homol'].str.strip()
pptos_faltantes['pieza'] = pptos_faltantes['Servicio'].str.slice(-2)
pptos_faltantes['pieza'] = pptos_faltantes['pieza'].str.strip()
pptos_faltantes['fecha_inicio'] = time.strftime("%d/%m/%Y")

pptos_faltantes = pptos_faltantes[['Clínica', 'Fecha Pres._x', 'Nº Historia', 'Presupuesto', 
                                   'Servicio', 'Semana', 'Cantidad Presupuestada','Doctor_x', 
                                   'Explicado_x', 'ttmto_homol','pieza', 'fecha_inicio']]

print('Numero de tratamientos faltantes por crono:',pptos_faltantes.count()[0])
pptos_faltantes.to_csv('pptos_faltantes.csv',index = False)

pptos_faltantes.set_index(['Clínica','Nº Historia','Servicio'])
pptos_faltantes

Numero de tratamientos faltantes por crono: 190


,Clínica,Fecha Pres._x,Nº Historia,Presupuesto,Servicio,Semana,Cantidad Presupuestada,Doctor_x,Explicado_x,ttmto_homol,pieza,fecha_inicio
0,Dentix Calle 90,22/09/2018,BG025054,Nº 4848 - Carlos Julio Alba Martinez,RETENEDOR FIJO 3X3 EN ALAMBRE. Pieza: 11,0.0,1.0,JOHANNA LISSETE MENDOZA DUARTE,ALISSON SARAY MORA CASTAÑEDA,RETENEDOR FIJO 3X3 EN ALAMBRE,11,25/09/2018
1,Dentix Calle 90,22/09/2018,BG025054,Nº 4848 - Carlos Julio Alba Martinez,RETENEDOR FIJO 3X3 EN ALAMBRE. Pieza: 31,0.0,1.0,JOHANNA LISSETE MENDOZA DUARTE,ALISSON SARAY MORA CASTAÑEDA,RETENEDOR FIJO 3X3 EN ALAMBRE,31,25/09/2018
2,Dentix Calle 90,22/09/2018,BG025054,Nº 4848 - Carlos Julio Alba Martinez,RETENEDOR ESSIX O ESTETICOS. Pieza: 11,0.0,1.0,JOHANNA LISSETE MENDOZA DUARTE,ALISSON SARAY MORA CASTAÑEDA,RETENEDOR ESSIX O ESTETICOS,11,25/09/2018
3,Dentix Calle 90,22/09/2018,BG058253,Nº 4850 - CARLOS JULIO ALBA,RETENEDOR FIJO 3X3 EN ALAMBRE. Pieza: 11,0.0,1.0,JOHANNA LISSETE MENDOZA DUARTE,ALISSON SARAY MORA CASTAÑEDA,RETENEDOR FIJO 3X3 EN ALAMBRE,11,25/09/2018
4,Dentix Calle 90,22/09/2018,BG058253,Nº 4850 - CARLOS JULIO ALBA,RETENEDOR FIJO 3X3 EN ALAMBRE. Pieza: 31,0.0,1.0,JOHANNA LISSETE MENDOZA DUARTE,ALISSON SARAY MORA CASTAÑEDA,RETENEDOR FIJO 3X3 EN ALAMBRE,31,25/09/2018
5,Dentix Calle 90,22/09/2018,BG058253,Nº 4850 - CARLOS JULIO ALBA,RETENEDOR ESSIX O ESTETICOS. Pieza: 11,0.0,1.0,JOHANNA LISSETE MENDOZA DUARTE,ALISSON SARAY MORA CASTAÑEDA,RETENEDOR ESSIX O ESTETICOS,11,25/09/2018
6,Dentix Calle 90,22/09/2018,BG058253,Nº 4851 - CARLOS JULIO ALBA,DETARTRAJE POR ARCADA.,0.0,1.0,LAURA ALEJANDRA RONCANCIO GARCIA,ALISSON SARAY MORA CASTAÑEDA,DETARTRAJE,,25/09/2018
7,Dentix Calle 90,22/09/2018,BG058253,Nº 4851 - CARLOS JULIO ALBA,PROFILAXIS. Pieza: 11,0.0,1.0,LAURA ALEJANDRA RONCANCIO GARCIA,ALISSON SARAY MORA CASTAÑEDA,PROFILAXIS,11,25/09/2018
8,Dentix Calle 90,22/09/2018,BG056466,Nº 4854 - Yobany Garcia Franco,ALARGAMIENTO CORONAL (POR PIEZA).,0.0,1.0,LAURA ALEJANDRA RONCANCIO GARCIA,ALISSON SARAY MORA CASTAÑEDA,ALARGAMIENTO CORONAL (,,25/09/2018
9,Dentix Calle 90,22/09/2018,BG056466,Nº 4854 - Yobany Garcia Franco,RESINA FOTOPOLIMERIZADO 1 SUP POSTERIOR Y ANTERIOR. Pieza: 15,0.0,1.0,LAURA ALEJANDRA RONCANCIO GARCIA,ALISSON SARAY MORA CASTAÑEDA,RESINA FOTOPOLIMERIZADO 1 SUP POSTERIOR Y ANTERIOR,15,25/09/2018


In [7]:
fecha_inicio=pptos_faltantes.set_index('Nº Historia')
fecha_inicio=fecha_inicio[['Presupuesto','fecha_inicio']]
fecha_inicio.reset_index(inplace=True)
fecha_inicio

fecha_inicio.drop_duplicates('Nº Historia', inplace=True)
fecha_inicio
fecha_inicio.to_csv('fecha_inicio.csv',index=False)